# vTTS + CosyVoice3 Kaggle 테스트

**CosyVoice3**: FunAudioLLM의 제로샷 다국어 TTS
- 9개 언어 지원 (한국어, 영어, 중국어, 일본어, 광동어, 스페인어, 프랑스어, 독일어, 포르투갈어)
- Zero-shot voice cloning
- 18+ 중국어 방언 지원

---

## 📋 단계별 가이드

1. **의존성 설치** - vTTS + CosyVoice 패키지
2. **서버 시작** - vTTS 서버 백그라운드 실행
3. **참조 오디오** - Supertonic으로 참조 음성 생성
4. **음성 합성** - CosyVoice로 제로샷 클로닝
5. **정리** - 서버 종료

## 1️⃣ 의존성 설치 (한 번만 실행)

CosyVoice 요구사항:
- `numpy==1.26.4` (필수!)
- `onnxruntime-gpu==1.18.0` (필수!)
- `torch==2.3.1`, `torchaudio==2.3.1`
- `transformers==4.51.3`

**⚠️ 중요**: 설치 후 **커널 재시작 필수**!

In [ ]:
%%bash
# 핵심 의존성 설치 (CosyVoice는 vTTS에 내장됨!)
echo "🔧 vTTS + CosyVoice 설치 중..."
echo ""

echo "[1/5] numpy==1.26.4 설치..."
pip uninstall -y numpy >/dev/null 2>&1
pip install --force-reinstall --no-cache-dir --ignore-installed numpy==1.26.4

echo "[2/5] onnxruntime-gpu==1.18.0 설치..."
pip uninstall -y onnxruntime onnxruntime-gpu >/dev/null 2>&1
pip install --force-reinstall --no-cache-dir --ignore-installed onnxruntime-gpu==1.18.0

echo "[3/5] PyTorch 2.3.1+cu121 설치..."
pip install --force-reinstall --no-cache-dir --ignore-installed \
    torch==2.3.1 torchaudio==2.3.1 \
    --index-url https://download.pytorch.org/whl/cu121

echo "[4/5] CosyVoice 의존성 설치..."
pip install --force-reinstall --no-cache-dir --ignore-installed \
    transformers==4.51.3 openai-whisper HyperPyYAML conformer \
    diffusers modelscope wetext x-transformers inflect

echo "[5/5] vTTS 설치 (CosyVoice 내장!)..."
pip install git+https://github.com/bellkjtt/vTTS.git --no-deps
pip install fastapi uvicorn httpx pydantic pydantic-settings click loguru soundfile huggingface-hub

echo ""
echo "✅ 설치 완료!"
echo "⚠️  지금 '런타임 > 세션 다시 시작'을 클릭하세요!"

In [ ]:
## ⚠️ 중요: 커널 재시작!

위 셀 실행 후 **반드시 커널을 재시작**하세요!

**Kaggle**: Runtime → Restart session
**Colab**: 런타임 → 세션 다시 시작

## ⚠️ 커널 재시작 후 여기서부터 실행!

위 셀 실행 후 **반드시 커널을 재시작**하고 아래부터 계속 진행하세요.

In [ ]:
# 버전 검증 (커널 재시작 후 실행!)
import numpy as np
import onnxruntime as ort
import torch

print("🔍 버전 확인\n" + "="*40)
print(f"numpy: {np.__version__}")
print(f"onnxruntime: {ort.__version__}")
print(f"torch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

# CosyVoice 내장 모듈 확인
try:
    from vtts.engines._cosyvoice.cli.cosyvoice import AutoModel
    print("\n✅ CosyVoice 내장 모듈: OK")
except ImportError as e:
    print(f"\n❌ CosyVoice 내장 모듈: {e}")

# 버전 검증
assert np.__version__ == "1.26.4", f"❌ numpy 버전 오류: {np.__version__}"
assert ort.__version__ == "1.18.0", f"❌ onnxruntime 버전 오류: {ort.__version__}"
print("\n✅ 모든 버전 확인 완료!")

# CosyVoice + vTTS API 테스트 (Kaggle)

**목표**: CosyVoice3 Zero-shot TTS를 vTTS API로 테스트

---

## 사전 요구사항
- Kaggle GPU 활성화 필수! (Settings → Accelerator → GPU T4 x2)
- 예상 시간: 설치 ~10분, 테스트 ~5분

## 1. uv 설치 (빠르고 정확한 패키지 관리자)

In [ ]:
# uv 설치
import subprocess
import sys

print("📦 uv 설치 중...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "uv"], check=True)
print("   ✅ uv 설치 완료!\n")

## 2. CosyVoice 설치 (~10분)

In [ ]:
# CosyVoice 설치 (uv 사용!)
import os
import subprocess
import sys

print("🔧 CosyVoice 환경 설정 중...")
print("   ⚠️ 이 과정은 10분 정도 소요됩니다.\n")

# CosyVoice 저장소 클론
print("   [1/4] CosyVoice 저장소 클론...")
COSYVOICE_PATH = os.path.expanduser("~/.vtts/CosyVoice")
os.makedirs(os.path.dirname(COSYVOICE_PATH), exist_ok=True)

if not os.path.exists(COSYVOICE_PATH):
    result = subprocess.run(
        ["git", "clone", "--depth", "1", "https://github.com/FunAudioLLM/CosyVoice.git", COSYVOICE_PATH],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        print(f"      ✅ 클론 완료: {COSYVOICE_PATH}")
    else:
        print(f"      ❌ 실패: {result.stderr}")
else:
    print(f"      ✅ 이미 존재: {COSYVOICE_PATH}")

# CosyVoice 의존성 설치
print("\n   [2/4] CosyVoice 의존성 설치 (uv 사용)...")
req_file = os.path.join(COSYVOICE_PATH, "requirements.txt")
subprocess.run(["uv", "pip", "install", "-r", req_file, "--system"], check=True)
print("      ✅ 완료!")

# 핵심 패키지 정확한 버전 강제 설치
print("\n   [3/4] 핵심 패키지 강제 설치...")
subprocess.run([
    "uv", "pip", "install", "--reinstall", "--system",
    "numpy==1.26.4", "torch==2.3.1", "torchaudio==2.3.1",
    "--index-url", "https://download.pytorch.org/whl/cu121"
], check=True)
subprocess.run([
    "uv", "pip", "install", "--reinstall", "--system",
    "onnxruntime-gpu==1.18.0", "transformers==4.51.3", "openai-whisper"
], check=True)
print("      ✅ numpy, torch, onnxruntime, transformers, whisper 설치 완료!")

# vTTS 설치
print("\n   [4/4] vTTS 설치...")
subprocess.run(["uv", "pip", "install", "--system", "--no-deps",
               "vtts[supertonic] @ git+https://github.com/bellkjtt/vTTS.git"], check=True)
subprocess.run(["uv", "pip", "install", "--system",
               "fastapi", "uvicorn", "httpx", "pydantic>=2.0",
               "loguru", "soundfile", "huggingface-hub", "click"], check=True)
print("      ✅ vTTS 설치 완료!")

# 검증
print("\n   [검증]")
import numpy as np
import torch
import onnxruntime as ort
print(f"      ✅ numpy: {np.__version__}")
print(f"      ✅ torch: {torch.__version__}")
print(f"      ✅ onnxruntime: {ort.__version__}")
print(f"      ✅ CUDA: {torch.cuda.is_available()}")
try:
    import whisper
    print("      ✅ whisper: OK")
except:
    print("      ❌ whisper: NOT FOUND")

sys.path.insert(0, COSYVOICE_PATH)
os.environ["COSYVOICE_PATH"] = COSYVOICE_PATH
print(f"\n✅ 설치 완료!")

## 3. 참조 오디오 생성 (Supertonic)

In [ ]:
# Supertonic으로 참조 오디오 생성
from pathlib import Path
from vtts.engines.supertonic import SupertonicEngine
from vtts.engines.base import EngineTTSRequest

print("🎙️ 참조 오디오 생성 중...\n")

engine = SupertonicEngine(model_id="Supertone/supertonic-2", device="cuda")
engine.load_model()

ref_dir = Path("./reference_audio")
ref_dir.mkdir(exist_ok=True)

# 한국어
ko_text = "안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다."
ko_path = ref_dir / "reference_ko.wav"
ko_request = EngineTTSRequest(text=ko_text, voice="F1", language="ko")
ko_audio = engine.synthesize(ko_request)
engine.save_audio(ko_audio, str(ko_path))
print(f"✅ 한국어: {ko_path}")

# 영어
en_text = "Hello, this is a reference audio for voice cloning."
en_path = ref_dir / "reference_en.wav"
en_request = EngineTTSRequest(text=en_text, voice="F1", language="en")
en_audio = engine.synthesize(en_request)
engine.save_audio(en_audio, str(en_path))
print(f"✅ 영어: {en_path}")

engine.unload_model()
print("\n✅ 완료!")

## 4. vTTS 서버 시작

In [ ]:
# vTTS 서버 시작
import subprocess
import time
import httpx

print("🚀 vTTS 서버 시작 중...\n")

server_process = subprocess.Popen(
    ["vtts", "serve", "FunAudioLLM/Fun-CosyVoice3-0.5B-2512",
     "--host", "0.0.0.0", "--port", "8000", "--device", "cuda"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1
)

print("서버 시작 대기 중 (최대 60초)...")
for i in range(60):
    try:
        response = httpx.get("http://localhost:8000/health", timeout=1.0)
        if response.status_code == 200:
            print(f"\n✅ 서버 준비 완료! ({i+1}초)")
            print("   API: http://localhost:8000/v1")
            break
    except:
        pass
    time.sleep(1)
    if i % 10 == 0:
        print(f"   대기 중... {i+1}s")
else:
    print("\n❌ 시간 초과!")

## 5. CosyVoice Zero-Shot TTS 테스트

In [ ]:
# Zero-Shot TTS 테스트
from vtts import VTTSClient
import IPython.display as ipd
from pathlib import Path

client = VTTSClient(base_url="http://localhost:8000")

print("🎙️ CosyVoice Zero-Shot TTS\n" + "="*60)

# 한국어
print("\n[1] 한국어")
ko_path = Path("./reference_audio/reference_ko.wav").absolute()
ko_audio = client.tts(
    text="이것은 코지보이스를 사용한 한국어 음성 클로닝 테스트입니다.",
    model="FunAudioLLM/Fun-CosyVoice3-0.5B-2512",
    language="ko",
    reference_audio=str(ko_path),
    reference_text="안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다."
)
print(f"✅ 완료! Duration: {len(ko_audio.audio)/ko_audio.sample_rate:.2f}s")
ko_audio.save("cosyvoice_korean.wav")
display(ipd.Audio(ko_audio.audio, rate=ko_audio.sample_rate))

# 영어
print("\n[2] 영어")
en_path = Path("./reference_audio/reference_en.wav").absolute()
en_audio = client.tts(
    text="This is an English voice cloning test using CosyVoice.",
    model="FunAudioLLM/Fun-CosyVoice3-0.5B-2512",
    language="en",
    reference_audio=str(en_path),
    reference_text="Hello, this is a reference audio for voice cloning."
)
print(f"✅ 완료! Duration: {len(en_audio.audio)/en_audio.sample_rate:.2f}s")
en_audio.save("cosyvoice_english.wav")
display(ipd.Audio(en_audio.audio, rate=en_audio.sample_rate))

print("\n" + "="*60)
print("✅ 테스트 완료!")

## 6. 정리 (서버 종료)

In [ ]:
# 서버 종료
print("🛑 서버 종료 중...")
server_process.terminate()
server_process.wait(timeout=5)
print("✅ 서버 종료 완료!")